In [64]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 17 01:14:06 2022

@author: vkedu
"""
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.model_selection import cross_val_score, train_test_split

In [65]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='nDK3erqrsighqUE5UEOFDu-ZyuiImZ6U4QoP2_2Ri2Zu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'carresalevaluepredictionibmklncei-donotdelete-pr-x5jzl2jrfwjnzx'
object_key = 'autos1.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,Unnamed: 0,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,...,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,...,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,...,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,...,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,...,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,...,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [66]:
df[df.seller != 'gewerblich']
df=df.drop('seller',1)

In [67]:
df[df.offerType != 'Gesuch']
df = df.drop('offerType',1)

In [68]:
df=df[(df.powerPS>50)&(df.powerPS<900)]
df=df[(df.yearOfRegistration >= 1950)&(df.yearOfRegistration<2017)]
df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'], axis='columns',inplace=True)

In [69]:
new_df=df.copy()
new_df=new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])

In [70]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'), inplace=True)
new_df.fuelType.replace(('benzin','andere','elecktro'),('petrol','others','electic'), inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'), inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'), inplace=True)


In [71]:
new_df=new_df[(new_df.price>=100)&(new_df.price<=150000)]

In [72]:
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df['gearbox'].fillna(value='not-declared',inplace=True)
new_df['vehicleType'].fillna(value='not-declared',inplace=True)
new_df['model'].fillna(value='not-declared',inplace=True)

In [73]:
new_df.to_csv("autos_preprocessed.csv")

In [74]:
l=['gearbox','notRepairedDamage','fuelType','vehicleType','model','brand']
m={}
for i in l:
    m[i]=LabelEncoder()
    m[i].fit(new_df[i])
    tr=m[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'),m[i].classes_)
    new_df.loc[:,i+'_labels']=pd.Series(tr,index=new_df.index)

In [75]:
l2=new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration'] + [x+"_labels" for x in l]]

In [76]:
Y=l2.iloc[:,0].values
X=l2.iloc[:,1:].values

In [77]:
Y=Y.reshape(-1,1)

In [78]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.3, random_state=3)

In [79]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [80]:
r=RandomForestRegressor(n_estimators=1000,max_depth=10,random_state=34)

In [81]:
r.fit(X_train,np.ravel(Y_train,order='C'))
y_pred=r.predict(X_test)
y_pred

array([9937.11993609, 1157.33268937, 2739.06719477, ..., 2124.91777611,
       2955.66758818, 1207.44132337])

In [82]:
!pip install ibm_watson_machine_learning

In [83]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"DceKrEN9EBsmpXev8xO21lXYhIohOV5BPm4DVs72adUW"
}
client=APIClient(wml_credentials)

In [84]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [85]:
space_uid=guid_from_space_name(client,'carresale model')
print(space_uid)

2c9c0aec-de8e-4fa1-8dd6-9f48044b8de2


In [86]:
client.set.default_space(space_uid)

'SUCCESS'

In [87]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [88]:
software_spec_uid = client.software_specifications.get_uid_by_name('runtime-22.1-py3.9')
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [89]:
model_details=client.repository.store_model(model=r,meta_props={
    client.repository.ModelMetaNames.NAME:"resale_model",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [90]:
model_id

'8480c953-e0a8-47e7-b573-be0d6370b164'

In [91]:
X_train[0]

array([  2002,    116, 150000,     10,      1,      0,      1,      1,
          155,     10])

In [92]:
r.predict([[2002,116, 150000,10,1,0,1,1,155,10]])

array([2602.23092201])